<a href="https://colab.research.google.com/github/alecseiterr/safe_city/blob/main/Taranov_Anatoly/Data_preparation/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9E%D0%B1%D1%8A%D0%B5%D0%B4%D0%B8%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2_%D1%81%D0%B5%D0%B3%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%86%D0%B8%D1%8E_label%2Bimage_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np

# Пути к папкам с изображениями и разметкой
images_dir = "images"
labels_dir = "labels"
output_dir = "output"

# Создаем папку для сохранения результатов, если она не существует
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Получаем список файлов с разметкой в папке "labels"
label_files = os.listdir(labels_dir)

for label_file in label_files:
    # Извлекаем имя файла без расширения
    base_name = os.path.splitext(label_file)[0]

    # Считываем изображение
    img_path = os.path.join(images_dir, base_name + ".jpg")
    img = cv2.imread(img_path)

    # Считываем разметку из файла .txt
    label_path = os.path.join(labels_dir, label_file)
    with open(label_path, "r") as file:
        lines = file.readlines()

    mask = np.zeros_like(img, dtype=np.uint8)

    for line in lines:
        points = line.strip().split()

        # Проверяем, что у нас есть хотя бы две координаты
        if len(points) >= 4:
            # Определяем цвет в зависимости от класса
            class_color = (0, 255, 0) if int(points[0]) == 0 else (0, 0, 255)
            # Преобразование координат сегментации
            points = [(int(float(points[i]) * img.shape[1]), int(float(points[i + 1]) * img.shape[0])) for i in range(1, len(points), 2)]

            # Создание маски сегментации
            if len(points) > 2:  # Для избежания ошибки при недостаточном количестве точек
                pts = np.array(points, np.int32)
                pts = pts.reshape((-1, 1, 2))
                cv2.fillPoly(mask, [pts], class_color)

    # Объединяем изображение с маской
    result = cv2.addWeighted(img, 1, mask, 0.5, 0)

    # Сохраняем изображение с сегментацией в папку "output"
    output_path = os.path.join(output_dir, base_name + ".jpg")
    cv2.imwrite(output_path, result)

    print(f"Преобразование завершено для файла: {label_file}")

print("Все файлы обработаны.")
